In [23]:
import pandas as pd
import numpy as np
import re
import os
import pymorphy3
import fasttext
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import warnings
import compress_fasttext
import requests
from gensim.models import Word2Vec

warnings.filterwarnings('ignore')

# Инициализация морфолизатора
morph = pymorphy3.MorphAnalyzer()

# Функция нормализации текста (определяем тут, чтобы не зависеть от внешних файлов)
def normalize_string(text):
    if not isinstance(text, str):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    # Лемматизация
    res = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(res)



# Словарь для конвертации тегов pymorphy в теги Word2Vec (Universal Dependencies)
morph_map = {
    'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADJ',
    'INFN': 'VERB', 'PRTF': 'ADJ', 'PRTS': 'ADJ', 'GRND': 'VERB',
    'NUMR': 'NUM', 'ADVB': 'ADV', 'NPRO': 'PRON',
    'PRED': 'ADV', 'PREP': 'ADP', 'CONJ': 'CCONJ'
}

def normalize_and_tag(text):
    if not isinstance(text, str): return []
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    
    tagged_words = []
    for word in words:
        p = morph.parse(word)[0]
        lemma = p.normal_form
        pos = p.tag.POS
        
        # Маппинг тегов
        ud_pos = morph_map.get(pos, pos)
        if ud_pos is None: ud_pos = 'NOUN' # Fallback
        
        # Формат: слово_ТЕГ (фильм_NOUN)
        tagged_words.append(f"{lemma}_{ud_pos}")
    return tagged_words

print("Библиотеки импортированы, функция нормализации готова.")

Библиотеки импортированы, функция нормализации готова.


In [3]:
# Загрузка данных
try:
    real_data = pd.read_csv('normalized.csv')
    test_data = pd.read_csv('harderTest.csv')
    print(f"Загружено фильмов: {real_data.shape[0]}")
    print(f"Загружено тестов: {test_data.shape[0]}")
except FileNotFoundError:
    print("Ошибка: Файлы normalized.csv или harderTest.csv не найдены!")

# Заполнение пропусков
real_data['overview'] = real_data['overview'].fillna('')

# 1. TF-IDF Векторизация
print("Создание TF-IDF матрицы...")
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 5))
tfidf_matrix = vectorizer.fit_transform(real_data['overview'])
print("TF-IDF готов.")

Загружено фильмов: 2000
Загружено тестов: 20
Создание TF-IDF матрицы...
TF-IDF готов.


In [17]:
# 1. Подготовка данных для Word2Vec (список списков слов)
print("Подготовка текстов для Word2Vec...")
# Нормализуем все описания
sentences = [normalize_string(text).split() for text in real_data['overview']]

# 2. Обучение модели
print("Обучение Word2Vec (локально)...")
# vector_size=100 - размер вектора
# window=5 - окно контекста
# min_count=1 - учитывать даже слова, встречающиеся 1 раз (важно для маленьких датасетов)
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
print("Word2Vec обучен!")

# Сохраним модель, чтобы бот мог её подхватить
w2v_model.save("local_word2vec.model")

# 3. Функция векторизации предложения (усредение векторов слов)
def get_w2v_vector(text):
    words = normalize_string(text).split()
    if not words: return np.zeros(100)
    
    vecs = []
    for word in words:
        if word in w2v_model.wv:
            vecs.append(w2v_model.wv[word])
    
    if not vecs: return np.zeros(100)
    return np.mean(vecs, axis=0)

# 4. Векторизация всей базы
print("Векторизация базы фильмов через Word2Vec...")
w2v_vectors = np.array([get_w2v_vector(t) for t in real_data['overview']])

# 5. Гибридный поиск (TF-IDF * Word2Vec)
def search_hybrid(query, top_k=1):
    norm = normalize_string(query)
    
    # TF-IDF сходство
    q_tfidf = vectorizer.transform([norm])
    sim_tfidf = cosine_similarity(q_tfidf, tfidf_matrix).flatten()
    
    # Word2Vec сходство
    q_w2v = get_w2v_vector(norm).reshape(1, -1)
    sim_w2v = cosine_similarity(q_w2v, w2v_vectors).flatten()
    
    # Гибрид
    hybrid_score = sim_tfidf * sim_w2v
    
    return real_data.iloc[hybrid_score.argsort()[::-1][:top_k]]['movie'].values

# Оценка точности
correct = 0
for i, row in test_data.iterrows():
    target = normalize_string(row['relevant_movie']).strip()
    preds = search_hybrid(row['query'], top_k=1)
    if target in [normalize_string(p).strip() for p in preds]:
        correct += 1

print(f"Точность Hybrid (Local Word2Vec * TF-IDF) @1: {correct / len(test_data):.2f}")

Подготовка текстов для Word2Vec...
Обучение Word2Vec (локально)...
Word2Vec обучен!
Векторизация базы фильмов через Word2Vec...
Точность Hybrid (Local Word2Vec * TF-IDF) @1: 0.45


In [25]:
print("Загрузка E5 (Small)...")
e5_model = SentenceTransformer('intfloat/multilingual-e5-small')

# Префиксы для E5 обязательны!
passages = ["passage: " + normalize_string(t) for t in real_data['overview']]
print("Кодирование базы через E5...")
e5_embeddings = e5_model.encode(passages, normalize_embeddings=True, show_progress_bar=True)

def search_e5(query, top_k=1):
    q_text = "query: " + normalize_string(query)
    q_emb = e5_model.encode([q_text], normalize_embeddings=True)
    scores = cosine_similarity(q_emb, e5_embeddings).flatten()
    return real_data.iloc[scores.argsort()[::-1][:top_k]]['movie'].values

# Оценка
correct_e5 = 0
for i, row in test_data.iterrows():
    target = normalize_string(row['relevant_movie']).strip()
    preds = search_e5(row['query'], top_k=1)
    if target in [normalize_string(p).strip() for p in preds]:
        correct_e5 += 1

print(f"Точность E5 Vector Search @1: {correct_e5 / len(test_data):.2f}")

Загрузка E5 (Small)...
Кодирование базы через E5...


Batches: 100%|██████████| 63/63 [2:09:54<00:00, 123.72s/it]   


Точность E5 Vector Search @1: 0.50


In [24]:
import zipfile

from gensim.models import KeyedVectors
# Ссылка на модель (RusVectores: GeoWAC, CBOW, 300 dim)
MODEL_URL = "http://vectors.nlpl.eu/repository/20/180.zip"
ARCHIVE_FILE = "180.zip"
MODEL_FILE = "model.bin" # Внутри архива он так называется


def normalize_simple(text):
    return ' '.join([x.split('_')[0] for x in normalize_and_tag(text)])

def download_and_extract_w2v():
    if not os.path.exists(MODEL_FILE):
        print("Скачивание Word2Vec (около 600 Мб)...")
        r = requests.get(MODEL_URL, stream=True)
        with open(ARCHIVE_FILE, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024*1024):
                if chunk: f.write(chunk)
        
        print("Распаковка архива...")
        with zipfile.ZipFile(ARCHIVE_FILE, 'r') as zip_ref:
            zip_ref.extractall(".")
        
        print("Готово.")
    else:
        print("Модель уже скачана.")

download_and_extract_w2v()

print("Загрузка модели в память (gensim)...")
# load_word2vec_format загружает стандартные бинарные файлы w2v
w2v_model = KeyedVectors.load_word2vec_format(MODEL_FILE, binary=True)
print("Модель Word2Vec загружена!")

# Векторизация
def get_w2v_vector(text):
    tagged_words = normalize_and_tag(text)
    if not tagged_words: return np.zeros(300)
    
    vecs = []
    for word in tagged_words:
        if word in w2v_model:
            vecs.append(w2v_model[word])
        else:
            # Fallback: пробуем без тега или с другим тегом
            simple_word = word.split('_')[0]
            if f"{simple_word}_NOUN" in w2v_model:
                vecs.append(w2v_model[f"{simple_word}_NOUN"])
                
    if not vecs: return np.zeros(300)
    return np.mean(vecs, axis=0)

print("Векторизация базы фильмов...")
w2v_vectors = np.array([get_w2v_vector(t) for t in real_data['overview']])

# Гибридный поиск
def search_hybrid(query, top_k=1):
    norm_simple = normalize_simple(query)
    
    # 1. TF-IDF
    q_tfidf = vectorizer.transform([norm_simple])
    sim_tfidf = cosine_similarity(q_tfidf, tfidf_matrix).flatten()
    
    # 2. Word2Vec
    q_w2v = get_w2v_vector(query).reshape(1, -1)
    sim_w2v = cosine_similarity(q_w2v, w2v_vectors).flatten()
    
    # 3. Hybrid
    hybrid_score = sim_tfidf * sim_w2v
    return real_data.iloc[hybrid_score.argsort()[::-1][:top_k]]['movie'].values

# Оценка
correct = 0
for i, row in test_data.iterrows():
    target = normalize_simple(row['relevant_movie']).strip()
    preds = search_hybrid(row['query'], top_k=1)
    if target in [normalize_simple(p).strip() for p in preds]:
        correct += 1
print(f"Точность Hybrid (Pretrained W2V * TF-IDF) @1: {correct / len(test_data):.2f}")

Модель уже скачана.
Загрузка модели в память (gensim)...
Модель Word2Vec загружена!
Векторизация базы фильмов...
Точность Hybrid (Pretrained W2V * TF-IDF) @1: 0.40
